In [ ]:
# =====================================================================
# Approach 2 (variant): INFERENCE on a BUBBLE CSV
# Converts p(non-bubble) → p(bubble)=1-p(non-bubble); prints summary.
# =====================================================================

import torch, torch.nn as nn, numpy as np, pandas as pd, warnings, os, time, math
warnings.filterwarnings("ignore")

# Colab detection
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ---- Encoders (must match training)
class EncoderBiLSTM(nn.Module):
    def __init__(self, in_dim, emb=128):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, emb, num_layers=2, bidirectional=True, batch_first=True)
        self.fc   = nn.Linear(emb*2, emb)
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        h = torch.cat([h[-2], h[-1]], dim=1)
        return nn.functional.normalize(self.fc(h), dim=1)

class SinusoidalPE(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2)*(-math.log(10000.0)/d_model))
        pe[:, 0::2] = torch.sin(pos * div); pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe.unsqueeze(0))
    def forward(self, x): return x + self.pe[:, :x.size(1)]

class EncoderTransformer(nn.Module):
    def __init__(self, in_dim, emb=128, nhead=4, num_layers=2, dropout=0.1, pool="last"):
        super().__init__()
        self.input = nn.Linear(in_dim, emb)
        layer = nn.TransformerEncoderLayer(d_model=emb, nhead=nhead, batch_first=True,
                                           dropout=dropout, norm_first=True)
        self.enc = nn.TransformerEncoder(layer, num_layers=num_layers)
        self.pos = SinusoidalPE(emb); self.pool = pool
        self.cls = nn.Parameter(torch.zeros(1,1,emb)) if pool == "cls" else None
    def forward(self, x):
        h = self.input(x)
        if self.cls is not None:
            cls = self.cls.expand(x.size(0), -1, -1); h = torch.cat([cls, h], dim=1)
        h = self.enc(self.pos(h))
        if self.pool == "cls":   z = h[:, 0, :]
        elif self.pool == "mean": z = h.mean(dim=1)
        else:                    z = h[:, -1, :]
        return nn.functional.normalize(z, dim=1)

class NonBubbleDetector(nn.Module):
    def __init__(self, in_dim, emb=128, kind="bilstm", pool="last", nhead=4, num_layers=2, dropout=0.1):
        super().__init__()
        self.encoder = EncoderBiLSTM(in_dim, emb) if kind=="bilstm" else \
                       EncoderTransformer(in_dim, emb, nhead, num_layers, dropout, pool)
        self.classifier = nn.Sequential(
            nn.Linear(emb, 64), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(64, 32), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(32, 1), nn.Sigmoid()
        )
    def forward(self, x):
        z = self.encoder(x); p_non = self.classifier(z)
        return z, p_non.squeeze()
    @torch.no_grad()
    def get_nonbubble_probability(self, x):
        _, p = self.forward(x); return p

def get_bubble_risk_label(p_bub):
    if p_bub >= 0.8: return "🔴 Very High Bubble Risk"
    if p_bub >= 0.6: return "🟠 High Bubble Risk"
    if p_bub >= 0.4: return "🟡 Moderate Bubble Risk"
    if p_bub >= 0.2: return "🟢 Low Bubble Risk"
    return "🔵 Very Low Bubble Risk"

def calculate_probs(csv_path, model, scalers, info):
    df = pd.read_csv(csv_path, parse_dates=["Date"])
    if "PPIACO" in df.columns and "PPI" not in df.columns:
        df.rename(columns={"PPIACO":"PPI"}, inplace=True)
    need_cols, macro_cols, dow_cols = scalers["need_cols"], scalers["macro_cols"], scalers["dow_cols"]
    df_clean = df.dropna(subset=need_cols).reset_index(drop=True)
    window = info.get("window", 24)
    if len(df_clean) < window:
        print(f"❗ Warning: only {len(df_clean)} rows (need {window}). Using all rows.")
        window = len(df_clean)
        if window == 0: return [], []
    Xm = scalers["sc_macro"].transform(df_clean[macro_cols]).astype("float32")
    Xd = scalers["sc_dow"].transform(df_clean[dow_cols]).astype("float32")
    p_non_list, p_bub_list = [], []
    for t in range(len(df_clean) - window + 1):
        seq = np.hstack([Xm[t:t+window], Xd[t:t+window]])
        seq = torch.tensor(seq).unsqueeze(0).to(DEVICE)
        p_non = float(NonBubbleDetector.get_nonbubble_probability(model, seq).cpu().item())
        p_bub = 1.0 - p_non
        p_non_list.append(p_non); p_bub_list.append(p_bub)
    return p_non_list, p_bub_list

# ---- Load package
if IN_COLAB:
    from google.colab import files
    print("📦 Upload 'nonbubble_model_package.pth' (trained on 3 NON-BUBBLE)")
    up = files.upload(); package_path = next(iter(up))
else:
    package_path = "nonbubble_model_package.pth"

package = torch.load(package_path, map_location=DEVICE, weights_only=False)
cfg, scalers = package["model_config"], package["scalers"]
assert cfg.get("predicts") == "non_bubble", "Loaded package is not a non-bubble model."
print(f"✅ Loaded | encoder={cfg['encoder_kind']} | emb={cfg['emb']} | pool={cfg['pool']} | trained_on={cfg.get('train_prototypes', [])}")

# Build model
model = NonBubbleDetector(in_dim=cfg.get("in_dim",6),
                          emb=cfg["emb"],
                          kind=cfg["encoder_kind"],
                          pool=cfg.get("pool","last"),
                          nhead=cfg.get("transformer",{}).get("nhead",4),
                          num_layers=cfg.get("transformer",{}).get("num_layers",2),
                          dropout=cfg.get("transformer",{}).get("dropout",0.1)).to(DEVICE)
model.load_state_dict(package["model_state_dict"]); model.eval()

# ---- Upload ONE bubble CSV (session k of 4)
if IN_COLAB:
    print("\n📂 Upload ONE BUBBLE CSV to evaluate:")
    up2 = files.upload(); csv_path = next(iter(up2))
else:
    csv_path = "YOUR_BUBBLE.csv"

t0 = time.time()
p_non, p_bub = calculate_probs(csv_path, model, scalers, cfg)
dt = (time.time() - t0)*1000

print("\n" + "="*70)
print(f"🔎 File: {os.path.basename(csv_path)}")
if not p_bub:
    print("❌ Not enough clean rows to analyze.")
else:
    mean_non, latest_non = float(np.mean(p_non)), float(p_non[-1])
    mean_bub, latest_bub = float(np.mean(p_bub)), float(p_bub[-1])
    trend = "N/A"
    if len(p_bub) > 1:
        slope = np.polyfit(range(len(p_bub)), p_bub, 1)[0]
        trend = "Increasing ↗️" if slope > 0.01 else ("Decreasing ↘️" if slope < -0.01 else "Stable ↔️")
    print(f"📈 Mean p(non-bubble): {mean_non:.4f}  →  Mean p(bubble)=1-p: {mean_bub:.4f}")
    print(f"📉 Latest p(non-bubble): {latest_non:.4f} → Latest p(bubble)=1-p: {latest_bub:.4f}")
    print(f"💬 Interpretation (bubble): {get_bubble_risk_label(latest_bub)}")
    print(f"📊 Bubble-Prob Trend:       {trend}")
    print(f"⏱️ Processing Time:         {dt:.2f} ms")
print("="*70)

📦 Upload 'nonbubble_model_package.pth' (trained on 3 NON-BUBBLE)


Saving nonbubble_model_package.pth to nonbubble_model_package (4).pth
✅ Loaded | encoder=transformer | emb=128 | pool=last | trained_on=['1962.01-1965.12', '1975.01-1980.01', '1987.11-1995.12']

📂 Upload ONE BUBBLE CSV to evaluate:
